In [ ]:
import sys

In [ ]:
from selenium import webdriver
import pandas as pd
import numpy as np
from time import sleep

In [ ]:
#open a Chrome session
driver = webdriver.Chrome('/Applications/chromedriver')

In [ ]:
#open all of Christie's jewelry archives
driver.get('https://www.christies.com/departments/Jewellery-33-1.aspx?pagesection=results#results')

#CLOSE SIGN UP POP UP IF IT EVER APPEARS (write code to handle this)

In [ ]:
view_all = driver.find_element_by_id('divViewAll')
view_all.click()

In [ ]:
#lists of auction links, auction names, and auction dates and locations
urls = []
names = []
dates_locs = [] #this only collects locations when on inspect mode. Fix this.


auction_results = driver.find_elements_by_id('Calendar-Results-Sales')


for auction in auction_results:
    urls.append(auction.find_element_by_css_selector('a').get_attribute('href'))
    dates_locs.append(auction.find_element_by_class_name('col-md-2').text)
    names.append(auction.find_element_by_class_name('image-description--title').text)

In [ ]:
#create dictionary with above lists
auctions = {key: (v1, v2) for key, v1, v2 in zip(urls, dates_locs, names)}

In [ ]:
print(len(auctions))

In [ ]:
#a few auction webpages do not follow the main general template, and will not successfully get scraped
#let's record these

failed_scrap = []

#inialize empty dictionary that will contain: 
#key - item name and description
#value - item price estimate, item price realized, item auction information 

dic = {}



#loop through auction_urls 
for auction in auctions:
    
    auc_info = auctions[auction]
    print(auc_info[0])
    print(auc_info[1])
    print(auc_info)
    driver.get(auction)
    sleep(15)
    
    try:
        #check this load all
        load_all = driver.find_element_by_id("loadAllUpcomingPast")
        load_all.click()
        sleep(20)
        
        #list of links for every lot in current auction
        lots = driver.find_elements_by_class_name('image-preview-container')
        lot_urls = []
        for lot in lots:
            lot_urls.append(lot.find_element_by_css_selector('a').get_attribute('href')) 
            
        #fill dic    
        for lot_url in lot_urls:
            try:
                driver.get(lot_url)
                sleep(10)
                descrip = driver.find_element_by_id("main_center_0_lblLotDescription")
                key = descrip.text
                price_realized = driver.find_element_by_id('main_center_0_lblPriceRealizedPrimary')
                dic[key] = [price_realized.text]
                price_estimated = driver.find_element_by_id('main_center_0_lblPriceEstimatedPrimary')
                dic[key].append(price_estimated.text)
                #need to add auction date_loc and name
                
                dic[key].append(auc_info[0])
                dic[key].append(auc_info[1])
            except:  
                continue
            
    except:
        #record failed scrapes
        failed_scrap.append(auction)
        continue
    

In [ ]:
print(len(dic))
print(failed_scrap)

In [ ]:
#create pandas dataframe from dic
df = pd.DataFrame.from_dict(dic, orient='index', columns = ['price_realized', 'price_estimated', 'date_loc', 'auc_name'])

In [ ]:
df = df.reset_index()
df = df.rename(columns={"index": 'name/descrip'})
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv(r'/Users/katieross/Desktop/auction_datasets/test1.csv')